In [ ]:
* Understanding Encoder-Decoder Sequence to Sequence Model   
https://towardsdatascience.com/understanding-encoder-decoder-sequence-to-sequence-model-679e04af4346

# Load the data

In [ ]:
# wget http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz

In [1]:
import tensorflow as tf

In [13]:
!dir ..\

 Volume in drive D has no label.
 Volume Serial Number is 7AB0-BEFF

 Directory of D:\work\work_git\ml-playground

03/29/2020  05:52 PM    <DIR>          .
03/29/2020  05:52 PM    <DIR>          ..
03/18/2020  09:34 PM    <DIR>          autoencoder
03/29/2020  05:52 PM    <DIR>          dataset
03/16/2020  09:59 PM    <DIR>          lecture_homework
03/11/2020  06:11 PM             4,514 load_test_autoencoder.py
03/18/2020  09:01 PM    <DIR>          logs
03/18/2020  11:53 PM    <DIR>          output
03/18/2020  08:06 PM             2,377 README.md
03/10/2020  06:19 PM             5,660 restore_training.py
03/16/2020  09:59 PM    <DIR>          tf_dataset
03/29/2020  05:43 PM    <DIR>          tf_tutorial
03/16/2020  09:59 PM    <DIR>          training_plot
03/09/2020  10:18 PM             2,998 train_conv_autoencoder.py
03/10/2020  06:19 PM             6,703 train_denoising_autoencoder.py
03/18/2020  08:06 PM             5,907 train_fan_autoencoder.py
03/19/2020  09:38 PM             

In [28]:
# Load data from a dataset comprising lines 
# from one or more text files.
lines = tf.data.TextLineDataset('..\dataset\CBTest\data\cbt_train.txt')

In [29]:
for line in lines.take(3):
    print(line)

tf.Tensor(b'_BOOK_TITLE_ : Andrew_Lang___Prince_Prigio.txt.out', shape=(), dtype=string)
tf.Tensor(b'CHAPTER I. -LCB- Chapter heading picture : p1.jpg -RCB- How the Fairies were not Invited to Court .', shape=(), dtype=string)
tf.Tensor(b'Once upon a time there reigned in Pantouflia a king and a queen .', shape=(), dtype=string)


In [19]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [30]:
# Filter Out Title Lines First 
def is_title(x):
    return tf.strings.regex_full_match(x, "_BOOK_TITLE_.*")

In [31]:
# Then We Remove All Punctuation
punctuation = r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']'
def remove_punc(x):
    return tf.strings.regex_replace(x, punctuation, ' ')

In [40]:
def remove_extra_spaces(x):
    return tf.strings.join(tf.strings.split(x), ' ')

In [45]:
def make_lower(x):
    return tf.strings.lower(x)

In [32]:
lines = lines.filter(lambda x: not is_title(x))

In [33]:
for line in lines.take(3):
    print(line)

tf.Tensor(b'CHAPTER I. -LCB- Chapter heading picture : p1.jpg -RCB- How the Fairies were not Invited to Court .', shape=(), dtype=string)
tf.Tensor(b'Once upon a time there reigned in Pantouflia a king and a queen .', shape=(), dtype=string)
tf.Tensor(b'With almost everything else to make them happy , they wanted one thing : they had no children .', shape=(), dtype=string)


In [46]:
lines = lines.map(lambda x: remove_punc(x))
for line in lines.take(3):
    print(line)

tf.Tensor(b'CHAPTER I   LCB  Chapter heading picture   p1 jpg  RCB  How the Fairies were not Invited to Court  ', shape=(), dtype=string)
tf.Tensor(b'Once upon a time there reigned in Pantouflia a king and a queen  ', shape=(), dtype=string)
tf.Tensor(b'With almost everything else to make them happy   they wanted one thing   they had no children  ', shape=(), dtype=string)


In [43]:
# Then We Remove All Extra Spaces Created By The Previous FN
# lines = lines.map(lambda x: remove_extra_spaces(x))
# for line in lines.take(3):
#     print(line)

In [47]:
# Then We Turn All The Uppercase Letters into Lowercase Letters
lines = lines.map(lambda x: make_lower(x))
for line in lines.take(3):
    print(line)

tf.Tensor(b'chapter i   lcb  chapter heading picture   p1 jpg  rcb  how the fairies were not invited to court  ', shape=(), dtype=string)
tf.Tensor(b'once upon a time there reigned in pantouflia a king and a queen  ', shape=(), dtype=string)
tf.Tensor(b'with almost everything else to make them happy   they wanted one thing   they had no children  ', shape=(), dtype=string)


In [48]:
# Get words from lines
words = lines.map(tf.strings.split)
wordsets = words.unbatch().batch(11)

for row in wordsets.take(3):
    print(row)

tf.Tensor(
[b'chapter' b'i' b'lcb' b'chapter' b'heading' b'picture' b'p1' b'jpg'
 b'rcb' b'how' b'the'], shape=(11,), dtype=string)
tf.Tensor(
[b'fairies' b'were' b'not' b'invited' b'to' b'court' b'once' b'upon' b'a'
 b'time' b'there'], shape=(11,), dtype=string)
tf.Tensor(
[b'reigned' b'in' b'pantouflia' b'a' b'king' b'and' b'a' b'queen' b'with'
 b'almost' b'everything'], shape=(11,), dtype=string)


In [52]:
wordsets

<BatchDataset shapes: (None,), types: tf.string>

# Label the data

In [75]:
def get_example_label(row):
    example = tf.strings.reduce_join(row[:-1], separator=' ')
#     print(example)
    example = tf.expand_dims(example, axis=0)
#     print(example)
    label = row[-1:]
    return example, label

In [76]:
for row in wordsets.take(1):
    print('row:',row)
    example, label = get_example_label(row)
    print('exmaple:',example)
    print('label',label)

row: tf.Tensor(
[b'chapter' b'i' b'lcb' b'chapter' b'heading' b'picture' b'p1' b'jpg'
 b'rcb' b'how' b'the'], shape=(11,), dtype=string)
exmaple: tf.Tensor([b'chapter i lcb chapter heading picture p1 jpg rcb how'], shape=(1,), dtype=string)
label tf.Tensor([b'the'], shape=(1,), dtype=string)


In [77]:
# get_example_label is a simple fn to split wordsets into examples and labels
# First ten words are the example and last word is the label
data = wordsets.map(get_example_label)
data = data.shuffle(1000)

In [78]:
for row in data.take(1):
    print(row)

(<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'majesty said prigio you must permit me to correct your'],
      dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'policy'], dtype=object)>)


In [79]:
data

<ShuffleDataset shapes: ((1,), (None,)), types: (tf.string, tf.string)>

# Preproces the data

## vectorize

In [53]:
vocab_size = 5000

In [54]:
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    output_sequence_length=10)

In [59]:
vectorize_layer.adapt(lines.batch(64))

In [60]:
vectorize_layer.get_vocabulary()[:5]

[b'the', b'and', b'to', b'a', b'of']

In [61]:
vectorize_layer.get_vocabulary()[-5:]

[b'repay', b'rejoice', b'reasonable', b'rail', b'quoted']

## batch and 

In [80]:
for batch in data.batch(3).take(1):
    print(batch[0])
    print(vectorize_layer(batch[0]))

tf.Tensor(
[[b'suit in a moment these were provided he bathed dressed']
 [b'he would have liked to say do n t you']
 [b'they were lying then he ran downstairs and walked out']], shape=(3, 1), dtype=string)
tf.Tensor(
[[1102   11    5  280  210   43 4758    8    1  942]
 [   8   45   35  523    4  138   37   30   28   13]
 [  31   43  767   58    8  301 2451    3  442   47]], shape=(3, 10), dtype=int64)


# seq2seq 2.0

In [82]:
!pip install tensorflow_addons

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [83]:
import tensorflow_addons as tfa

In [84]:
print(tfa.__version__)

0.8.3


In [85]:
print(dir(tfa.seq2seq))

['AttentionMechanism', 'AttentionWrapper', 'AttentionWrapperState', 'BahdanauAttention', 'BahdanauMonotonicAttention', 'BaseDecoder', 'BasicDecoder', 'BasicDecoderOutput', 'BeamSearchDecoder', 'BeamSearchDecoderOutput', 'BeamSearchDecoderState', 'CustomSampler', 'Decoder', 'FinalBeamSearchDecoderOutput', 'GreedyEmbeddingSampler', 'InferenceSampler', 'LuongAttention', 'LuongMonotonicAttention', 'SampleEmbeddingSampler', 'Sampler', 'ScheduledEmbeddingTrainingSampler', 'ScheduledOutputTrainingSampler', 'SequenceLoss', 'TrainingSampler', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'attention_wrapper', 'basic_decoder', 'beam_search_decoder', 'decoder', 'dynamic_decode', 'gather_tree', 'gather_tree_from_array', 'hardmax', 'loss', 'monotonic_attention', 'safe_cumprod', 'sampler', 'sequence_loss', 'tile_batch']


In [86]:
MAX_VOCAB_SIZE = 5000
WINDOW_LENGTH = 11
class EncoderDecoder(tf.keras.Model):
    def __init__(self, 
                 max_features=MAX_VOCAB_SIZE, 
                 output_seq_len=WINDOW_LENGTH-1,  ## 
                 embedding_dims=200, 
                 rnn_units=512):
        
        super().__init__()
        
        self.max_features = max_features
        self.output_seq_len = output_seq_len
        self.embedding_dims = embedding_dims
        self.rnn_units = rnn_units
        
        # vecctorize
        self.vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
                max_tokens=max_features,
                output_sequence_length=10)
        # encoder
        self.encoder_embedding = tf.keras.layers.Embedding(
            max_features + 1, embedding_dims)
        self.lstm_layer = tf.keras.layers.LSTM(rnn_units, return_state=True)
        # decoder 
        self.decoder_embedding = tf.keras.layers.Embedding(
            max_features + 1, embedding_dims)
        
        
        # ---------------- MAYBE NOT NECESSARY ----------------
        # Sampler (for use during training)
        # This was not shown during the talk but it is pretty obvious
        sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        # ref the https://github.com/tensorflow/addons/issues/603
#         train_sampler = tfa.seq2seq.TrainingSampler()
#         inference_sampler = tfa.seq2seq.InferenceSampler()
        
        # This was not shown during the talk but is required... 
        # This is my best guess
        decoder_cell = tf.keras.layers.LSTMCell(units=self.rnn_units)
        # ---------------- MAYBE NOT NECESSARY ----------------
        
        
        projection_layer = tf.keras.layers.Dense(max_features)
        
        
        self.decoder = tfa.seq2seq.BasicDecoder(
            decoder_cell, sampler, output_layer=projection_layer)

#         self._train_decoder = tfa.seq2seq.BasicDecoder(
#                 decoder_cell, train_sampler , output_layer=projection_layer)
    
#         self._inference_decoder = tfa.seq2seq.BasicDecoder(
#                 decoder_cell, inference_sampler, output_layer=projection_layer)
        
        # pay attention
        self. attention = tf.keras.layers.Attention()
        
#     def call(inputs, training=None):
#         if training is None:
#             training = tf.keras.backend.learning_phase()
        
#         if training:
#             # training stuff
#             pass
#         else:
#             # inference stuff
#             pass

    def train_step(self, data):
        """ Overwrite built-in train_step method

        Args:
            data (tuple): The example (ten `words`), and the label (one `word`)

        Returns:
            Metric results for all passed metrics
        """

        # Split data into example (x) and label (y)
        x, y = data[0], data[1]

        # Vectorize the example words (x)
        x = self.vectorize_layer(x)

        # Vectorize the labels
        # This will by default pad the output to 10 ... but we only need the
        # first entry (the true label not the useless padding)
        y = self.vectorize_layer(y)[:, 0:1]

        # Convert our label into a one-hot encoding based on the max number of
        # features that we will be using for our model
        y_one_hot = tf.one_hot(y, self.max_features)

        # Everything within GradientTape is recorded 
        # for later automatic differentiation
        with tf.GradientTape() as tape:

            # --- <ENCODER STUFF> ---

            # Transform the example utilizing the encoder embedding
            inputs = self.encoder_embedding(x)

            # Get the encoder outputs and state by 
            # utilizing the encoder (lstm_layer)
            #   - encoder_outputs : [max_time, batch_size, num_units]
            #   - encoder_state   : [state_h, state_c]
            #       * state_h --- The Hidden State
            #       * state_c --- The Cell   State
            encoder_outputs, state_h, state_c = self.lstm_layer(inputs)

            # --- </ENCODER STUFF> ---

            # --- <ATTN STUFF> ---

            # Pass the encoder outputs and hidden state allowing us
            # to track the intermediate state coming out of the encoder layers
            attn_output = self.attention([encoder_outputs, state_h])
            attn_output = tf.expand_dims(attn_output, axis=1)

            # --- </ATTN STUFF> ---

            # --- <DECODER STUFF> ---

            # ??? Create an empty embedding ???
            targets = self.decoder_embedding(tf.zeros_like(y))

            # Concat the output of the attention layer to the last axis
            # of the empty targets embedding
            concat_output = tf.concat([targets, attn_output], axis=-1)

            # Predict the targets using the state from the encoder
            outputs, _, _ = \
                self.decoder(concat_output, initial_state=[state_h, state_c])

            # --- </DECODER STUFF> ---

        # Automatically differeniate utilizing the loss and trainable variables
        gradients = tape.gradient(loss, trainable_variables)

        # Collect the outputs so that they can be optimized
        self.optimizer.apply_gradients(zip(gradients, trainable_variables))

        # Update the metric state prior to return
        self.compiled_metrics.update_state(y_one_hot, y_pred)

        return {m.name: m.result() for m in self.metrics}

In [87]:
model = EncoderDecoder()

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer="adam",
    metrics=["accuracy"])

model.vectorize_layer.adapt(lines.batch(256))



In [89]:
data

<ShuffleDataset shapes: ((1,), (None,)), types: (tf.string, tf.string)>

In [88]:
# ERROR OCCURS ON THIS LINE
model.fit(data.batch(256),
          epochs=45,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath='text_gen')])

AttributeError: 'NoneType' object has no attribute 'dtype'